In [33]:
import pandas as pd
import json


## Benchmark import

In [41]:
import pandas as pd
import ast

df_benchmark = pd.read_csv('benchmark.csv')
df_benchmark['Context'] = df_benchmark['Context'].apply(ast.literal_eval)

In [ ]:
display(df_benchmark)

# Mistral 7b-Instruct v0.3 raw

In [43]:
df = pd.read_csv('datasetsrag/Mistral-7b-instruct-v03-fp.csv', encoding='utf-8')

def extract_query_and_context(row):
    if pd.isna(row['output_intermediate_steps']):
        return pd.Series([None, None], index=['query', 'context'])
    
    try:
        steps = json.loads(row['output_intermediate_steps'])
        query = steps[0]['query']
        context_list = steps[1]['context']
        context = ", ".join([json.dumps(ctx,ensure_ascii=False) for ctx in context_list])
        return pd.Series([query, context], index=['query', 'context'])
    except (json.JSONDecodeError, KeyError, TypeError) as e:
        print(f"Error processing row: {row['output_intermediate_steps']} - {e}")
        return pd.Series([None, None], index=['query', 'context'])

# Appliquer la fonction à chaque ligne du DataFrame
df[['query', 'context']] = df.apply(extract_query_and_context, axis=1)

# Supprimer la colonne originelle
df.drop(columns=['output_intermediate_steps'], inplace=True)
df.drop(columns=['output_result'], inplace=True)

def extract_values(context_string):
    if pd.isna(context_string):
        return []  # Retourner une liste vide pour les valeurs NaN
    try:
        # Convertir la chaîne en une liste de dictionnaires
        context_list = json.loads(f'[{context_string}]')
        # Extraire les valeurs des dictionnaires
        values = [list(d.values())[0] for d in context_list]
        return values
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {context_string} - {e}")
        return []

# Appliquer la fonction à chaque ligne de la colonne 'context'
df['context_values'] = df['context'].apply(extract_values)

# Afficher le DataFrame transformé


In [44]:
import numpy as np

# Split df into 5 equal parts
dfs = np.array_split(df, 5)

# Assign each part to a separate variable
df1, df2, df3, df4, df5 = dfs

In [46]:
import ast


for df in [df1, df2, df3, df4, df5]:
    df.reset_index(drop=True, inplace=True)
    df['perfect_match'] = 0  # Initialiser la colonne avec des zéros

# Fonction pour vérifier les correspondances
def check_perfect_match(df, benchmark_col, test_col):
    for i in df.index:
        if i < len(benchmark_col) and i < len(test_col):
            if benchmark_col[i] == test_col[i]:
                df.at[i, 'perfect_match'] = 1

# Vérifier les correspondances pour chaque dataframe
check_perfect_match(df1, df_benchmark["Context"], df1["context_values"])
check_perfect_match(df2, df_benchmark["Context"], df2["context_values"])
check_perfect_match(df3, df_benchmark["Context"], df3["context_values"])
check_perfect_match(df4, df_benchmark["Context"], df4["context_values"])
check_perfect_match(df5, df_benchmark["Context"], df5["context_values"])

In [47]:
total_perfect_match = (
    df1['perfect_match'].sum() +
    df2['perfect_match'].sum() +
    df3['perfect_match'].sum() +
    df4['perfect_match'].sum() +
    df5['perfect_match'].sum()
)

percentage_perfect_match = (total_perfect_match / 125) * 100

print(f"total des match : {total_perfect_match} Pourcentage de perfect match: {percentage_perfect_match:.2f}%")


total des match : 105 Pourcentage de perfect match: 84.00%


# Mistral-7B-Instruct-v0.3 4Bit

In [49]:
df = pd.read_csv('datasetsrag/Mistral-7b-instruct-v03-4bit.csv', encoding='utf-8')

def extract_query_and_context(row):
    if pd.isna(row['output_intermediate_steps']):
        return pd.Series([None, None], index=['query', 'context'])
    
    try:
        steps = json.loads(row['output_intermediate_steps'])
        query = steps[0]['query']
        context_list = steps[1]['context']
        context = ", ".join([json.dumps(ctx,ensure_ascii=False) for ctx in context_list])
        return pd.Series([query, context], index=['query', 'context'])
    except (json.JSONDecodeError, KeyError, TypeError) as e:
        print(f"Error processing row: {row['output_intermediate_steps']} - {e}")
        return pd.Series([None, None], index=['query', 'context'])

# Appliquer la fonction à chaque ligne du DataFrame
df[['query', 'context']] = df.apply(extract_query_and_context, axis=1)

# Supprimer la colonne originelle
df.drop(columns=['output_intermediate_steps'], inplace=True)
df.drop(columns=['output_output'], inplace=True)

def extract_values(context_string):
    if pd.isna(context_string):
        return []  # Retourner une liste vide pour les valeurs NaN
    try:
        # Convertir la chaîne en une liste de dictionnaires
        context_list = json.loads(f'[{context_string}]')
        # Extraire les valeurs des dictionnaires
        values = [list(d.values())[0] for d in context_list]
        return values
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {context_string} - {e}")
        return []

# Appliquer la fonction à chaque ligne de la colonne 'context'
df['context_values'] = df['context'].apply(extract_values)

# Afficher le DataFrame transformé


In [188]:
import numpy as np

# Split df into 5 equal parts
dfs = np.array_split(df, 5)

# Assign each part to a separate variable
df1, df2, df3, df4, df5 = dfs

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes
pd.set_option('display.max_rows', None)     # Afficher toutes les lignes
pd.set_option('display.max_colwidth', None) # Ne pas tronquer le contenu des colonnes
pd.set_option('display.width', 1000)        # Ajuster la largeur d'affichage

display(df1)


In [129]:
import ast


for df in [df1, df2, df3, df4, df5]:
    df.reset_index(drop=True, inplace=True)
    df['perfect_match'] = 0  # Initialiser la colonne avec des zéros

# Fonction pour vérifier les correspondances
def check_perfect_match(df, benchmark_col, test_col):
    for i in df.index:
        if i < len(benchmark_col) and i < len(test_col):
            if benchmark_col[i] == test_col[i]:
                df.at[i, 'perfect_match'] = 1

# Vérifier les correspondances pour chaque dataframe
check_perfect_match(df1, df_benchmark["Context"], df1["context_values"])
check_perfect_match(df2, df_benchmark["Context"], df2["context_values"])
check_perfect_match(df3, df_benchmark["Context"], df3["context_values"])
check_perfect_match(df4, df_benchmark["Context"], df4["context_values"])
check_perfect_match(df5, df_benchmark["Context"], df5["context_values"])

In [132]:
for df in [df1, df2, df3, df4, df5]:
    df['exploitable'] = df['perfect_match']

In [130]:
total_perfect_match = (
    df1['perfect_match'].sum() +
    df2['perfect_match'].sum() +
    df3['perfect_match'].sum() +
    df4['perfect_match'].sum() +
    df5['perfect_match'].sum()
)

percentage_perfect_match = (total_perfect_match / 125) * 100

print(f"total des match : {total_perfect_match} Pourcentage de perfect match: {percentage_perfect_match:.2f}%")


total des match : 83 Pourcentage de perfect match: 66.40%


In [133]:
total_exploitable = (
    df1['exploitable'].sum() +
    df2['exploitable'].sum() +
    df3['exploitable'].sum() +
    df4['exploitable'].sum() +
    df5['exploitable'].sum()
)

# Diviser par 125 pour obtenir le pourcentage
percentage_exploitable = (total_exploitable / 125) * 100

# Afficher le pourcentage pour exploitable
print(f"Total des exploitable : {total_exploitable} Pourcentage de exploitable: {percentage_exploitable:.2f}%")


Total des exploitable : 83 Pourcentage de exploitable: 66.40%


# Mistral v0.3 4bit

In [144]:
import re
df = pd.read_csv('datasetsrag/Mistral7Bv034bit.csv', encoding='utf-8')

In [145]:
df.drop (columns=['input_schema'], inplace=True)

In [153]:
def extract_first_query(text):
    # Chercher les expressions "match" ou "MATCH" suivies par "return ... as ..." ou juste "return ..."
    match = re.search(r'(?i)(match.*?return\s+.*?\sas\s+\S+|match.*?return\s+\S+)', text, re.DOTALL)
    if match:
        return match.group(1)
    return ""

# Appliquer la fonction à la colonne 'output_text' pour créer la colonne 'first query'
df['first query'] = df['output_text'].apply(extract_first_query)

In [155]:
df.drop (columns=['output_text'], inplace=True)

In [158]:
driver = GraphDatabase.driver(url, auth=(username, password),database=db_name)

# Function to execute a query
def execute_query(query, parameters=None):
    try:
        with driver.session() as session:
            result = session.run(query, parameters)
            return result.value()
    except Exception as e:
        print(f"Erreur lors de l'exécution de la requête : {query}\nErreur : {e}")
        return None

def fetch_results(query):
    if query:
        return execute_query(query)
    return None

df['result'] = df['first query'].apply(fetch_results)

# Fermer la connexion au driver une fois terminé
driver.close()

Erreur lors de l'exécution de la requête : MATCH (n: Disease)-[:has_expression]->(m:Indication) WHERE apoc.text.clean(n.name_fr) = apoc.text.clean("Laryngite aiguë")  return m.name_fr UNION MATCH (n: Disease)-[:is_part_of]->(g: Group)-[:has_expression]->(m:Indication) WHERE apoc.text.clean(n.name_fr) = apoc.text.clean("Laryngite aiguë") return m.name_fr


Voici la question :
Quels sont les traitements des troubles schizo-affectifs ?

donnez uniquement la query sans utiliser une formule d'introduction

match (d:Disease)-[:is_treated_by]->(dci:DCI) where apoc.text.clean(d.name_fr) =  apoc.text.clean("Troubles schizo-affectifs")
    return dci.name_fr as Traitements
Erreur : {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'Voici': expected
  "!="
  "%"
  "*"
  "+"
  ","
  "-"
  "/"
  "::"
  "<"
  "<="
  "<>"
  "="
  "=~"
  ">"
  ">="
  "AND"
  "AS"
  "CALL"
  "CONTAINS"
  "CREATE"
  "DELETE"
  "DETACH"
  "ENDS"
  "FINISH"
  "FOREACH"
  "IN"
  "INSERT"
  "IS"
  "LIMIT

In [160]:
import numpy as np

# Split df into 5 equal parts
dfs = np.array_split(df, 5)

# Assign each part to a separate variable
df1, df2, df3, df4, df5 = dfs

In [163]:
import ast

#df_benchmark['Context'] = df_benchmark['Context'].apply(ast.literal_eval)
for df in [df1, df2, df3, df4, df5]:
    df.reset_index(drop=True, inplace=True)
    df['perfect_match'] = 0  # Initialiser la colonne avec des zéros

# Fonction pour vérifier les correspondances
def check_perfect_match(df, benchmark_col, test_col):
    for i in df.index:
        if i < len(benchmark_col) and i < len(test_col):
            if benchmark_col[i] == test_col[i]:
                df.at[i, 'perfect_match'] = 1

# Vérifier les correspondances pour chaque dataframe
check_perfect_match(df1, df_benchmark["Context"], df1["result"])
check_perfect_match(df2, df_benchmark["Context"], df2["result"])
check_perfect_match(df3, df_benchmark["Context"], df3["result"])
check_perfect_match(df4, df_benchmark["Context"], df4["result"])
check_perfect_match(df5, df_benchmark["Context"], df5["result"])

In [165]:
total_perfect_match = (
    df1['perfect_match'].sum() +
    df2['perfect_match'].sum() +
    df3['perfect_match'].sum() +
    df4['perfect_match'].sum() +
    df5['perfect_match'].sum()
)

percentage_perfect_match = (total_perfect_match / 125) * 100

print(f"total des match : {total_perfect_match} Pourcentage de perfect match: {percentage_perfect_match:.2f}%")

total des match : 25 Pourcentage de perfect match: 20.00%


# Meta-llama-3-8b-instruct

In [167]:
import re
df = pd.read_csv('datasetsrag/meta llama responses.csv', encoding='utf-8')
df.drop (columns=['input_schema'], inplace=True)
def extract_first_query(text):
    # Chercher les expressions "match" ou "MATCH" suivies par "return ... as ..." ou juste "return ..."
    match = re.search(r'(?i)(match.*?return\s+.*?\sas\s+\S+|match.*?return\s+\S+)', text, re.DOTALL)
    if match:
        return match.group(1)
    return ""

# Appliquer la fonction à la colonne 'output_text' pour créer la colonne 'first query'
df['first query'] = df['output_text'].apply(extract_first_query)

In [ ]:
driver = GraphDatabase.driver(url, auth=(username, password),database=db_name)

# Function to execute a query
def execute_query(query, parameters=None):
    try:
        with driver.session() as session:
            result = session.run(query, parameters)
            return result.value()
    except Exception as e:
        print(f"Erreur lors de l'exécution de la requête : {query}\nErreur : {e}")
        return None

def fetch_results(query):
    if query:
        return execute_query(query)
    return None

df['real_result'] = df['output_text'].apply(fetch_results)

# Fermer la connexion au driver une fois terminé
driver.close()

In [172]:
count_real_result = df["real_result"].notnull().sum()
print(f"Nombre d'occurrences non nulles dans df['real_result']: {count_real_result}")

Nombre d'occurrences non nulles dans df['real_result']: 0


In [173]:
import numpy as np

# Split df into 5 equal parts
dfs = np.array_split(df, 5)

# Assign each part to a separate variable
df1, df2, df3, df4, df5 = dfs

In [174]:
import ast

#df_benchmark['Context'] = df_benchmark['Context'].apply(ast.literal_eval)
for df in [df1, df2, df3, df4, df5]:
    df.reset_index(drop=True, inplace=True)
    df['perfect_match'] = 0  # Initialiser la colonne avec des zéros

# Fonction pour vérifier les correspondances
def check_perfect_match(df, benchmark_col, test_col):
    for i in df.index:
        if i < len(benchmark_col) and i < len(test_col):
            if benchmark_col[i] == test_col[i]:
                df.at[i, 'perfect_match'] = 1

# Vérifier les correspondances pour chaque dataframe
check_perfect_match(df1, df_benchmark["Context"], df1["result"])
check_perfect_match(df2, df_benchmark["Context"], df2["result"])
check_perfect_match(df3, df_benchmark["Context"], df3["result"])
check_perfect_match(df4, df_benchmark["Context"], df4["result"])
check_perfect_match(df5, df_benchmark["Context"], df5["result"])

In [175]:
total_perfect_match = (
    df1['perfect_match'].sum() +
    df2['perfect_match'].sum() +
    df3['perfect_match'].sum() +
    df4['perfect_match'].sum() +
    df5['perfect_match'].sum()
)

percentage_perfect_match = (total_perfect_match / 125) * 100

print(f"total des match : {total_perfect_match} Pourcentage de perfect match: {percentage_perfect_match:.2f}%")

total des match : 91 Pourcentage de perfect match: 72.80%


# Meta-llama-3-8b-instruct-4Bits

In [177]:
import re
df = pd.read_csv('datasetsrag/Meta-llama-3-8b-instruct4bit-dataset.csv', encoding='utf-8')
df.drop (columns=['input_schema'], inplace=True)
def extract_first_query(text):
    # Chercher les expressions "match" ou "MATCH" suivies par "return ... as ..." ou juste "return ..."
    match = re.search(r'(?i)(match.*?return\s+.*?\sas\s+\S+|match.*?return\s+\S+)', text, re.DOTALL)
    if match:
        return match.group(1)
    return ""

# Appliquer la fonction à la colonne 'output_text' pour créer la colonne 'first query'
df['first query'] = df['output_text'].apply(extract_first_query)

In [ ]:
driver = GraphDatabase.driver(url, auth=(username, password),database=db_name)

# Function to execute a query
def execute_query(query, parameters=None):
    try:
        with driver.session() as session:
            result = session.run(query, parameters)
            return result.value()
    except Exception as e:
        print(f"Erreur lors de l'exécution de la requête : {query}\nErreur : {e}")
        return None

def fetch_results(query):
    if query:
        return execute_query(query)
    return None

df['result'] = df['first query'].apply(fetch_results)

# Fermer la connexion au driver une fois terminé
driver.close()

In [182]:
import ast

#df_benchmark['Context'] = df_benchmark['Context'].apply(ast.literal_eval)
for df in [df1, df2, df3, df4, df5]:
    df.reset_index(drop=True, inplace=True)
    df['perfect_match'] = 0  # Initialiser la colonne avec des zéros

# Fonction pour vérifier les correspondances
def check_perfect_match(df, benchmark_col, test_col):
    for i in df.index:
        if i < len(benchmark_col) and i < len(test_col):
            if benchmark_col[i] == test_col[i]:
                df.at[i, 'perfect_match'] = 1

# Vérifier les correspondances pour chaque dataframe
check_perfect_match(df1, df_benchmark["Context"], df1["result"])
check_perfect_match(df2, df_benchmark["Context"], df2["result"])
check_perfect_match(df3, df_benchmark["Context"], df3["result"])
check_perfect_match(df4, df_benchmark["Context"], df4["result"])
check_perfect_match(df5, df_benchmark["Context"], df5["result"])

In [183]:
total_perfect_match = (
    df1['perfect_match'].sum() +
    df2['perfect_match'].sum() +
    df3['perfect_match'].sum() +
    df4['perfect_match'].sum() +
    df5['perfect_match'].sum()
)

percentage_perfect_match = (total_perfect_match / 125) * 100

print(f"total des match : {total_perfect_match} Pourcentage de perfect match: {percentage_perfect_match:.2f}%")

total des match : 91 Pourcentage de perfect match: 72.80%


In [185]:
df

,input_question,output_text,first query,result,real_result,perfect_match
0,Quelles sont les alternatives thérapeutiques pour l'anxiété généralisée ?,"match (d:Disease)-[:is_treated_by]->(dci:DCI) where apoc.text.clean(d.name_fr) = apoc.text.clean(""Anxiété généralisée"")\n return dci.name_fr as Alternatives thérapeutiques\n union match (d:Disease)-[:is_part_of]->(g:Group)-[:is_treated_by]->(dci:DCI) where apoc.text.clean(d.name_fr) = apoc.text.clean(""Anxiété généralisée"")\n return dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thé","match (d:Disease)-[:is_treated_by]->(dci:DCI) where apoc.text.clean(d.name_fr) = apoc.text.clean(""Anxiété généralisée"")\n return dci.name_fr as Alternatives","[prégabaline, escitalopram, venlafaxine, paroxétine, duloxétine]",None,1
1,Comment traiter les phobies sociales ?,"match (d:Disease)-[:is_treated_by]->(dci:DCI) where apoc.text.clean(d.name_fr) = apoc.text_clean(""phobies sociales"")\n return dci.name_fr as Traitements\n union match (d:Disease)-[:has_expression]->(m:Indication) where apoc.text.clean(d.name_fr) = apoc.text_clean(""phobies sociales"")\n return m.name_fr union match (d:Disease)-[:is_part_of]->(g:Group)-[:has_expression]->(m:Indication) where apoc.text.clean(d.name_fr) = apoc.text_clean(""phobies sociales"")\n return m.name_fr\n order by m.name_fr asc\n limit 10\n return distinct m.name_fr as Symptômes\n union match (d:Disease)-[:has_expression]->(m:Indication) where apoc.text.clean(d.name_fr) = apoc.text_clean(""phobies sociales"")\n return m.name_fr union match (d:Disease)-[:is_part_of]->(g:Group)-[:has_expression]->(m:Indication) where apoc.text.clean(d.name_fr) = apoc.text_clean(""phobies sociales"")\n return m.name_fr\n order by m.name_fr asc\n limit 10\n return distinct m.name_fr as Symptômes\n union match (d:Disease)-[:has_express

In [186]:
df1

,input_question,output_text,first query,result,real_result,perfect_match
0,Quelles sont les alternatives thérapeutiques pour l'anxiété généralisée ?,"match (d:Disease)-[:is_treated_by]->(dci:DCI) where apoc.text.clean(d.name_fr) = apoc.text.clean(""Anxiété généralisée"")\n return dci.name_fr as Alternatives thérapeutiques\n union match (d:Disease)-[:is_part_of]->(g:Group)-[:is_treated_by]->(dci:DCI) where apoc.text.clean(d.name_fr) = apoc.text.clean(""Anxiété généralisée"")\n return dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thérapeutiques\n order by dci.name_fr asc\n limit 10\n offset 0\n return distinct dci.name_fr as Alternatives thé","match (d:Disease)-[:is_treated_by]->(dci:DCI) where apoc.text.clean(d.name_fr) = apoc.text.clean(""Anxiété généralisée"")\n return dci.name_fr as Alternatives","[prégabaline, escitalopram, venlafaxine, paroxétine, duloxétine]",None,1
1,Comment traiter les phobies sociales ?,"match (d:Disease)-[:is_treated_by]->(dci:DCI) where apoc.text.clean(d.name_fr) = apoc.text_clean(""phobies sociales"")\n return dci.name_fr as Traitements\n union match (d:Disease)-[:has_expression]->(m:Indication) where apoc.text.clean(d.name_fr) = apoc.text_clean(""phobies sociales"")\n return m.name_fr union match (d:Disease)-[:is_part_of]->(g:Group)-[:has_expression]->(m:Indication) where apoc.text.clean(d.name_fr) = apoc.text_clean(""phobies sociales"")\n return m.name_fr\n order by m.name_fr asc\n limit 10\n return distinct m.name_fr as Symptômes\n union match (d:Disease)-[:has_expression]->(m:Indication) where apoc.text.clean(d.name_fr) = apoc.text_clean(""phobies sociales"")\n return m.name_fr union match (d:Disease)-[:is_part_of]->(g:Group)-[:has_expression]->(m:Indication) where apoc.text.clean(d.name_fr) = apoc.text_clean(""phobies sociales"")\n return m.name_fr\n order by m.name_fr asc\n limit 10\n return distinct m.name_fr as Symptômes\n union match (d:Disease)-[:has_express